# SNS Sentiment Analysis

Analyze social media texts and measure potential inflammatory / offensive language.

Install NLTK data if not already installed

In [1]:
import nltk, os

nltk.data.path.append('./nltk_data/')
if not os.path.exists('./nltk_data'):
    nltk.download('punkt', download_dir='./nltk_data/')
    nltk.download('stopwords', download_dir='./nltk_data/')